In [1]:
%pip install pandas
%pip install xgboost
%pip install sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
customers_data = pd.read_csv('data_sets/customers.csv')

# CHECK FOR NULL VALUES
customers_data.isnull().sum()

CUSTOMER_ID      0
x_customer_id    0
y_customer_id    0
dtype: int64

In [3]:
# USE x_terminal_id and y_terminal__id to calculate euclidean distance
customers_data['distance'] = ((customers_data['x_customer_id'] - customers_data['y_customer_id'])**2)**0.5

# DROP COLUMNS THAT ARE NOT NEEDED
customers_data = customers_data.drop(columns=['x_customer_id', 'y_customer_id'])

customers_data

,CUSTOMER_ID,distance
0,7894622031164826,84.885726
1,65009550197454,61.108141
2,5381378528001886,52.106019
3,344044685095332,54.379830
4,5281794686066290,39.704794
...,...,...
49995,2018185637039049,58.958321
49996,7838600820947425,5.662280
49997,1310171896045522,41.466918
49998,3346506901987756,30.413742


In [4]:
merchants_data = pd.read_csv('data_sets/merchants.csv')

# CHECK FOR NULL VALUES
merchants_data.isnull().sum()

MERCHANT_ID                           0
BUSINESS_TYPE                         0
MCC_CODE                              0
LEGAL_NAME                            0
FOUNDATION_DATE                       0
TAX_EXCEMPT_INDICATOR                 0
OUTLET_TYPE                           0
ACTIVE_FROM                           0
TRADING_FROM                          0
ANNUAL_TURNOVER_CARD                  0
ANNUAL_TURNOVER                       0
AVERAGE_TICKET_SALE_AMOUNT            0
PAYMENT_PERCENTAGE_FACE_TO_FACE       0
PAYMENT_PERCENTAGE_ECOM               0
PAYMENT_PERCENTAGE_MOTO               0
DEPOSIT_REQUIRED_PERCENTAGE           0
DEPOSIT_PERCENTAGE                    0
DELIVERY_SAME_DAYS_PERCENTAGE         0
DELIVERY_WEEK_ONE_PERCENTAGE          0
DELIVERY_WEEK_TWO_PERCENTAGE          0
DELIVERY_OVER_TWO_WEEKS_PERCENTAGE    0
dtype: int64

In [5]:
# DROP COLUMNS THAT ARE NOT NEEDED
merchants_data = merchants_data.drop(columns=['BUSINESS_TYPE', 'MCC_CODE', 'LEGAL_NAME', 'FOUNDATION_DATE',
                                                'TAX_EXCEMPT_INDICATOR', 'ACTIVE_FROM', 'TRADING_FROM', 'ANNUAL_TURNOVER_CARD',
                                                  'AVERAGE_TICKET_SALE_AMOUNT', 'DELIVERY_SAME_DAYS_PERCENTAGE',
                                                    'DELIVERY_WEEK_ONE_PERCENTAGE', 'DELIVERY_WEEK_TWO_PERCENTAGE', 
                                                    'DELIVERY_OVER_TWO_WEEKS_PERCENTAGE',
                                                    'PAYMENT_PERCENTAGE_FACE_TO_FACE','PAYMENT_PERCENTAGE_ECOM',
                                                    'PAYMENT_PERCENTAGE_MOTO'])

# CALCULATE DIFFERENCE IN DEPOSIT_REQUIRED_PERCENTAGE AND DEPOSIT_PERCENTAGE AND SAVE TO DEPOSIT_DIFF
merchants_data['DEPOSIT_DIFF'] = merchants_data['DEPOSIT_REQUIRED_PERCENTAGE'] - merchants_data['DEPOSIT_PERCENTAGE']

# DROP THE INDIVIDUAL DEPOSIT_PERCENTAGE AND DEPOSIT_REQUIRED_PERCENTAGE COLUMNS
merchants_data = merchants_data.drop(columns=['DEPOSIT_REQUIRED_PERCENTAGE', 'DEPOSIT_PERCENTAGE'])

merchants_data

,MERCHANT_ID,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
0,1b7557bb-208b-4fdc-ba91-eca98298541a,Ecommerce,351840227,0
1,6c948718-ea18-4d07-85d7-0916eb55e699,Ecommerce,411017547,0
2,f0151a49-7ac0-4258-93db-46bee2db6f66,Ecommerce,306887834,0
3,07089d03-0d63-4d24-a3f9-e651fda07cdc,Ecommerce,686209305,0
4,2c16629a-da48-4e69-9862-8fd521a53bbc,Ecommerce,384701644,0
...,...,...,...,...
30447,f58e8478-1566-4d1c-b0cb-b2106c8da3be,Face to Face and Ecommerce,304951,0
30448,68618d14-1e80-449d-95e3-6fa9e61901be,Face to Face and Ecommerce,735472,0
30449,e3cfd7f2-e2d1-40ea-b7b3-2583c57ecb3a,Face to Face and Ecommerce,645184,0
30450,3f8dff6f-d5b0-4963-99fa-3c64830fed23,Face to Face and Ecommerce,912396,0


In [6]:
terminals_data = pd.read_csv('data_sets/terminals.csv')

# CHECK FOR NULL VALUES
terminals_data.isnull().sum()

TERMINAL_ID       0
x_terminal_id     0
y_terminal__id    0
dtype: int64

In [7]:
# USE x_terminal_id and y_terminal__id to calculate euclidean distance
terminals_data['distance'] = ((terminals_data['x_terminal_id'] - terminals_data['y_terminal__id'])**2)**0.5

# DROP COLUMNS THAT ARE NOT NEEDED
terminals_data = terminals_data.drop(columns=['x_terminal_id', 'y_terminal__id'])

terminals_data

,TERMINAL_ID,distance
0,9754224,27.353181
1,37467861,40.981413
2,20678215,33.963044
3,12724021,26.240403
4,2989793,29.912286
...,...,...
4995,47999983,5.515555
4996,13129180,16.320659
4997,74645821,29.213743
4998,65145440,1.160258


In [8]:
transactions_test_data = pd.read_csv('data_sets/transactions_test.csv')

# CHECK FOR NULL VALUES
transactions_test_data.isnull().sum()

TX_ID                                        0
TX_TS                                        0
CUSTOMER_ID                                  0
TERMINAL_ID                                  0
TX_AMOUNT                                    0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT        0
TRANSACTION_CASHBACK_AMOUNT                  0
CARD_EXPIRY_DATE                             0
CARD_DATA                                    0
CARD_BRAND                                   0
TRANSACTION_TYPE                             0
TRANSACTION_STATUS                           0
FAILURE_CODE                             70009
FAILURE_REASON                           70009
TRANSACTION_CURRENCY                         0
CARD_COUNTRY_CODE                            0
MERCHANT_ID                                  1
IS_RECURRING_TRANSACTION                     0
ACQUIRER_ID                                  0
CARDHOLDER_AUTH_METHOD                       0
ID_JOIN                                      0
dtype: int64

In [9]:
# DROP COLUMNS
transactions_test_data = transactions_test_data.drop(columns=['TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA', 'CARD_EXPIRY_DATE',
                                                                  'TRANSACTION_STATUS', 'TRANSACTION_TYPE',
                                                                  'CARD_COUNTRY_CODE','TRANSACTION_CURRENCY', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_test_data = transactions_test_data.loc[:, (transactions_test_data != 0.0).any(axis=0)]

transactions_test_data

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN
0,09324d812ba7915c3f791e973db293ad50db70d8,4635360198955981,4641801,0.0,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0
1,a17bc798bacadbde6cd7928688e2453fa6023d67,3695434572951846,75652463,0.0,ebd8abbf-db80-4ac2-a83b-6b3642f7140a,False,ACQ3,Signature,16253.0
2,dee3efed3600425f60eee0e9d363f9bac226e12a,2021639056384826,45740229,0.0,faffd1ae-c962-4a11-b664-2ce691c23df8,False,ACQ3,Signature,20501.0
3,e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,5182312031642618,12372906,0.0,ef16b77c-1595-4af7-9a50-a0b9f8507bbf,False,ACQ3,Signature,450.0
4,b25981de8b5dcdc1c38de111a21312115c34199d,2904034201365313,61785600,0.0,b62e5e95-63fa-4fa8-b3d2-3eb03616208a,False,ACQ1,Signature,24900.0
...,...,...,...,...,...,...,...,...,...
71134,b8a998afbc7152514b1ae19d3e74c1566fb9547b,7090668360287420,76409951,0.0,a3c9dbaa-b003-429a-bf20-8df2036f3d95,False,ACQ4,Offline enciphered PIN and signature,5445.0
71135,dd1f46bf3fe4cbe806c000d33c9491595aad9706,8394492078987172,22410848,0.0,5b02c96f-87ab-49e5-bec0-58d284654fa6,False,ACQ4,Offline enciphered PIN and signature,20215.0
71136,ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,2115306202677133,45761959,0.0,6a8b7002-938a-4fba-829c-7f47a6d3838a,False,ACQ4,Offline enciphered PIN and signature,20895.0
71137,005c9942ba8e4e05508fb47d332cd5ca802512f7,555840843479473,80700449,0.0,e2de765a-bef0-4793-9517-cc647acc5060,False,ACQ4,Offline enciphered PIN and signature,20954.0


In [10]:
# CHECK FOR NULL VALUES AGAIN
transactions_test_data.isnull().sum()

TX_ID                          0
CUSTOMER_ID                    0
TERMINAL_ID                    0
TRANSACTION_CASHBACK_AMOUNT    0
MERCHANT_ID                    1
IS_RECURRING_TRANSACTION       0
ACQUIRER_ID                    0
CARDHOLDER_AUTH_METHOD         0
ID_JOIN                        0
dtype: int64

In [11]:
transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')

# CHECK FOR NULL VALUES
transactions_train_data.isnull().sum()

C:\Users\bbdnet2779\AppData\Local\Temp\ipykernel_2504\2073585214.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_train_data = pd.read_csv('data_sets/transactions_train.csv')


TX_ID                                          0
TX_TS                                          0
CUSTOMER_ID                                    0
TERMINAL_ID                                    0
TX_AMOUNT                                      0
TX_FRAUD                                       0
TRANSACTION_GOODS_AND_SERVICES_AMOUNT          0
TRANSACTION_CASHBACK_AMOUNT                    0
CARD_EXPIRY_DATE                               0
CARD_DATA                                      0
CARD_BRAND                                     0
TRANSACTION_TYPE                               0
TRANSACTION_STATUS                             0
FAILURE_CODE                             1049208
FAILURE_REASON                           1049208
TRANSACTION_CURRENCY                           0
CARD_COUNTRY_CODE                              0
MERCHANT_ID                                    0
IS_RECURRING_TRANSACTION                       0
ACQUIRER_ID                                    1
CARDHOLDER_AUTH_METH

In [12]:
# Clean data:
transactions_train_data = transactions_train_data.drop(columns=['TX_ID', 'TX_TS', 'TX_AMOUNT', 'FAILURE_CODE',
                                                                 'FAILURE_REASON', 'CARD_BRAND', 'CARD_DATA',
                                                                   'CARD_EXPIRY_DATE', 'TRANSACTION_STATUS', 'TRANSACTION_TYPE',
                                                                   'CARD_COUNTRY_CODE','TRANSACTION_CURRENCY', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT'])

# DROP COLUMN IF ALL VALUES ARE 0
transactions_train_data = transactions_train_data.loc[:, (transactions_train_data != 0.0).any(axis=0)]

# DROP ROW IF A VALUE IS NAN
transactions_train_data = transactions_train_data.dropna()

transactions_train_data

,CUSTOMER_ID,TERMINAL_ID,TX_FRAUD,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD
0,7323088226725338,55875360,0,0.0,136def43-2ef3-4bc9-a8c2-60598c316e9c,False,ACQ1,Online PIN
1,8627200351202456,31418810,0,0.0,a019e1d4-a3b3-479e-9747-a2ece27ff980,False,ACQ2,Signature
2,8983139483148479,84302882,0,0.0,f15d1eca-36f0-4f11-b17d-fe36475b021e,False,ACQ1,Online PIN
3,2016744441942099,26364082,0,0.0,7bd1dea7-72fa-41b3-8232-b315e1187b58,False,ACQ2,Offline plaintext PIN
4,6888146658050582,34582161,0,0.0,eee6cb9f-b64d-4710-8c36-1c2b426d7466,False,ACQ2,Offline plaintext PIN
...,...,...,...,...,...,...,...,...
1066434,2426772567661946,29332740,0,0.0,26edbb0d-e15c-4e05-a36a-edba7a80348e,False,ACQ2,Online PIN
1066435,5935188564996005,17853512,0,0.0,badaefe1-3194-4c8d-bf17-1170a7434b91,False,ACQ3,Offline enciphered PIN
1066436,1236547395079298,47342746,0,0.0,f5ea7e47-61b5-43a2-acaa-4ac66a0d527f,False,ACQ3,Online PIN
1066437,1023056874020037,35501969,0,0.0,6e9e0952-1b76-45d5-8693-a4423d0d80da,True,ACQ4,Online PIN


In [13]:
# CHECK FOR NULL VALUES AGAIN
transactions_train_data.isnull().sum()

CUSTOMER_ID                    0
TERMINAL_ID                    0
TX_FRAUD                       0
TRANSACTION_CASHBACK_AMOUNT    0
MERCHANT_ID                    0
IS_RECURRING_TRANSACTION       0
ACQUIRER_ID                    0
CARDHOLDER_AUTH_METHOD         0
dtype: int64

In [14]:
# MERGE TABLES FOR TRAINING DATA
train_data = pd.merge(transactions_train_data, merchants_data, on='MERCHANT_ID', how='left')

train_data

,CUSTOMER_ID,TERMINAL_ID,TX_FRAUD,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
0,7323088226725338,55875360,0,0.0,136def43-2ef3-4bc9-a8c2-60598c316e9c,False,ACQ1,Online PIN,Face to Face,23598,0
1,8627200351202456,31418810,0,0.0,a019e1d4-a3b3-479e-9747-a2ece27ff980,False,ACQ2,Signature,Face to Face,340735,-10
2,8983139483148479,84302882,0,0.0,f15d1eca-36f0-4f11-b17d-fe36475b021e,False,ACQ1,Online PIN,Face to Face and Ecommerce,707188,0
3,2016744441942099,26364082,0,0.0,7bd1dea7-72fa-41b3-8232-b315e1187b58,False,ACQ2,Offline plaintext PIN,Face to Face and Ecommerce,104869,0
4,6888146658050582,34582161,0,0.0,eee6cb9f-b64d-4710-8c36-1c2b426d7466,False,ACQ2,Offline plaintext PIN,Face to Face and Ecommerce,788913,0
...,...,...,...,...,...,...,...,...,...,...,...
1066434,2426772567661946,29332740,0,0.0,26edbb0d-e15c-4e05-a36a-edba7a80348e,False,ACQ2,Online PIN,Face to Face,166262,0
1066435,5935188564996005,17853512,0,0.0,badaefe1-3194-4c8d-bf17-1170a7434b91,False,ACQ3,Offline enciphered PIN,Face to Face and Ecommerce,398860,0
1066436,1236547395079298,47342746,0,0.0,f5ea7e47-61b5-43a2-acaa-4ac66a0d527f,False,ACQ3,Online PIN,Face to Face and Ecommerce,911287,0
1066437,1023056874020037,35501969,0,0.0,6e9e0952-1b76-45d5-8693-a4423d0d80da,True,ACQ4,Online PIN,Face to Face,425738,0


In [15]:
# PRINT ANY ENTRIES THAT ARE NA
print(train_data[train_data.isna().any(axis=1)])

Empty DataFrame
Columns: [CUSTOMER_ID, TERMINAL_ID, TX_FRAUD, TRANSACTION_CASHBACK_AMOUNT, MERCHANT_ID, IS_RECURRING_TRANSACTION, ACQUIRER_ID, CARDHOLDER_AUTH_METHOD, OUTLET_TYPE, ANNUAL_TURNOVER, DEPOSIT_DIFF]
Index: []


In [16]:
# MERGE TABLES FOR TESTING DATA
test_data = pd.merge(transactions_test_data, merchants_data, on='MERCHANT_ID', how='left')

test_data

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
0,09324d812ba7915c3f791e973db293ad50db70d8,4635360198955981,4641801,0.0,52f53dfb-6f2c-4584-8c82-aeafbcd7a02d,False,ACQ2,Signature,5637.0,Ecommerce,389840.0,0.0
1,a17bc798bacadbde6cd7928688e2453fa6023d67,3695434572951846,75652463,0.0,ebd8abbf-db80-4ac2-a83b-6b3642f7140a,False,ACQ3,Signature,16253.0,Face to Face,643907.0,0.0
2,dee3efed3600425f60eee0e9d363f9bac226e12a,2021639056384826,45740229,0.0,faffd1ae-c962-4a11-b664-2ce691c23df8,False,ACQ3,Signature,20501.0,Face to Face,560326.0,0.0
3,e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,5182312031642618,12372906,0.0,ef16b77c-1595-4af7-9a50-a0b9f8507bbf,False,ACQ3,Signature,450.0,Face to Face,502041340.0,0.0
4,b25981de8b5dcdc1c38de111a21312115c34199d,2904034201365313,61785600,0.0,b62e5e95-63fa-4fa8-b3d2-3eb03616208a,False,ACQ1,Signature,24900.0,Face to Face,576342.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71134,b8a998afbc7152514b1ae19d3e74c1566fb9547b,7090668360287420,76409951,0.0,a3c9dbaa-b003-429a-bf20-8df2036f3d95,False,ACQ4,Offline enciphered PIN and signature,5445.0,Face to Face and Ecommerce,711524.0,0.0
71135,dd1f46bf3fe4cbe806c000d33c9491595aad9706,8394492078987172,22410848,0.0,5b02c96f-87ab-49e5-bec0-58d284654fa6,False,ACQ4,Offline enciphered PIN and signature,20215.0,Ecommerce,815031.0,0.0
71136,ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,2115306202677133,45761959,0.0,6a8b7002-938a-4fba-829c-7f47a6d3838a,False,ACQ4,Offline enciphered PIN and signature,20895.0,Face to Face,70252.0,0.0
71137,005c9942ba8e4e05508fb47d332cd5ca802512f7,555840843479473,80700449,0.0,e2de765a-bef0-4793-9517-cc647acc5060,False,ACQ4,Offline enciphered PIN and signature,20954.0,Face to Face,614036.0,0.0


In [17]:
# PRINT ANY ENTRIES THAT ARE NA
test_data[test_data.isna().any(axis=1)]

,TX_ID,CUSTOMER_ID,TERMINAL_ID,TRANSACTION_CASHBACK_AMOUNT,MERCHANT_ID,IS_RECURRING_TRANSACTION,ACQUIRER_ID,CARDHOLDER_AUTH_METHOD,ID_JOIN,OUTLET_TYPE,ANNUAL_TURNOVER,DEPOSIT_DIFF
8201,ddaa070acea087eae360225e92c1609cea905e43,1513339476778402,77821795,0.0,NaN,False,ACQ3,Online PIN,0.0,NaN,NaN,NaN


In [18]:
# Separate target prediction
y = train_data.TX_FRAUD
X = train_data.drop(['TX_FRAUD' ,'CUSTOMER_ID',  'TERMINAL_ID','MERCHANT_ID'], axis=1)
test_data_TX_ID = test_data.TX_ID
X_test = test_data.drop(['CUSTOMER_ID',  'TERMINAL_ID','MERCHANT_ID', 'ID_JOIN', 'TX_ID'], axis=1)
test_data_TX_ID.head()

0    09324d812ba7915c3f791e973db293ad50db70d8
1    a17bc798bacadbde6cd7928688e2453fa6023d67
2    dee3efed3600425f60eee0e9d363f9bac226e12a
3    e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7
4    b25981de8b5dcdc1c38de111a21312115c34199d
Name: TX_ID, dtype: object

In [19]:
# Divide data into training data and validation data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
object_cols_test_data = [col for col in X_train.columns if X_train[col].dtype == "object"]

In [20]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols].astype(str)))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols].astype(str)))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols].astype(str)))

c:\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_cols_test.index = X_test.index

In [22]:
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols_test_data, axis=1)

In [23]:
# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

In [24]:
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)
OH_X_test.columns = OH_X_test.columns.astype(str)

In [25]:
OH_X_train.head()

,TRANSACTION_CASHBACK_AMOUNT,ANNUAL_TURNOVER,DEPOSIT_DIFF,0,1,2,3,4,5,6,...,8,9,10,11,12,13,14,15,16,17
586284,0.0,594342838,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1003473,0.0,245774,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
527600,0.0,16357,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
258650,0.0,430950,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
608642,0.0,341877,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [26]:
reg_model = XGBRegressor(eval_metric=mean_absolute_error, learning_rate=0.05, n_estimators=500)
reg_model.fit(OH_X_train, y_train, eval_set=[(OH_X_valid, y_valid)], early_stopping_rounds=5, verbose=False)

c:\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)
c:\Python311\Lib\site-packages\xgboost\data.py:52

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x000001377FA8A480>,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=500,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [27]:
reg_model.evals_result()

{'validation_0': OrderedDict([('rmse',
               [0.1581709986230937,
                0.15815415122440113,
                0.15813904254564945,
                0.15812575770346385,
                0.15811403879371924,
                0.1581026012331725,
                0.15809343846851567,
                0.15808496764650642,
                0.15807804098986364,
                0.15807173394425877,
                0.1580650172478926,
                0.15805981364645785,
                0.15805471852388883,
                0.1580510913706082,
                0.158048440095743,
                0.15804465037753637,
                0.15804128460735659,
                0.15803868248493366,
                0.15803604424497192,
                0.15803490646423693,
                0.15803276766785967,
                0.15803186294503044,
                0.15803198131512192,
                0.1580304672738566,
                0.15802969407238557,
                0.15802905329596306,
      

In [28]:
answers = reg_model.predict(OH_X_test)
answers

c:\Python311\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Python311\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Python311\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Python311\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categorical_dtype(dtype)


array([0.02383485, 0.02248139, 0.02205592, ..., 0.02243544, 0.02337503,
       0.02267178], dtype=float32)

In [29]:
output = pd.DataFrame({"TX_ID": test_data_TX_ID, "TX_FRAUD": answers})
output.set_index("TX_ID", inplace=True)
output

,TX_FRAUD
TX_ID,
09324d812ba7915c3f791e973db293ad50db70d8,0.023835
a17bc798bacadbde6cd7928688e2453fa6023d67,0.022481
dee3efed3600425f60eee0e9d363f9bac226e12a,0.022056
e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.022899
b25981de8b5dcdc1c38de111a21312115c34199d,0.023162
...,...
b8a998afbc7152514b1ae19d3e74c1566fb9547b,0.022249
dd1f46bf3fe4cbe806c000d33c9491595aad9706,0.023000
ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,0.022435


In [30]:
max_a = output.TX_FRAUD.max()
min_a = output.TX_FRAUD.min()
min_norm = 0
max_norm =1
output['TX_FRAUD'] = round((output.TX_FRAUD- min_a) *(max_norm - min_norm) / (max_a-min_a) + min_norm , 3)
output

,TX_FRAUD
TX_ID,
09324d812ba7915c3f791e973db293ad50db70d8,0.023
a17bc798bacadbde6cd7928688e2453fa6023d67,0.021
dee3efed3600425f60eee0e9d363f9bac226e12a,0.020
e0b665434d7945d6b170eb8da1b88d3fdfaaf3f7,0.022
b25981de8b5dcdc1c38de111a21312115c34199d,0.022
...,...
b8a998afbc7152514b1ae19d3e74c1566fb9547b,0.021
dd1f46bf3fe4cbe806c000d33c9491595aad9706,0.022
ddceeb33d5d85e1ca3851bc6e84141b8964a45cd,0.021


In [31]:
output.to_csv('submission.csv')